In [1]:
import pickle
import gym
import time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from run_test import *

import atari_head_dataset as ahd 
import utils
from LearnAtariGazeHumanTrajs import create_training_data, learn_reward, Net

Logging to /tmp/openai-2019-05-11-17-57-04-921416


In [2]:
use_gaze= True
env_type = "atari"
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
tf.set_random_seed(seed)
data_dir = "../data/atari-head/"
env_name="seaquest"
reward_model_path="learned_models/seaquest_test"
env_id = "SeaquestNoFrameskip-v4"

In [3]:
n_train = 10000 #number of pairs of trajectories to create
snippet_length = 50 #length of trajectory for training comparison
lr = 0.0001
weight_decay = 0.0
num_iter = 5 #num times through training data
l1_reg=0.0
stochastic = True

In [4]:
env = make_vec_env(env_id, 'atari', 1, seed,
                   wrapper_kwargs={
                       'clip_rewards':False,
                       'episode_life':False,
                   })

env = VecFrameStack(env, 4)
agent = PPO2Agent(env, env_type, stochastic)


In [5]:
dataset = ahd.AtariHeadDataset(env_name, data_dir)
demonstrations, learning_returns, learning_rewards, learning_gaze = utils.get_preprocessed_trajectories(env_name, dataset, data_dir, use_gaze)

env name:  seaquest
valid trials: ['240_RZ_9931101_Feb-11-18-29-57', '248_RZ_802972_Feb-21-20-37-22', '237_RZ_9656617_Feb-08-14-12-21', '212_RZ_7139876_Jan-10-11-09-22', '241_RZ_19306_Feb-12-18-57-06', '185_RZ_9437843_Jun-19-14-51-34', '235_RZ_9578441_Feb-07-16-28-09', '238_RZ_9659492_Feb-08-14-59-42', '197_RZ_3876283_Dec-03-16-37-07']
valid trial nums: [240, 248, 237, 212, 241, 185, 235, 238, 197]
traj length:  9
Max human score 445860
Min human score 45990
num non duplicate scores 9
len demos:  9
4263 4263 4263


In [6]:
training_obs, training_labels, training_gaze = create_training_data(demonstrations, learning_returns, learning_rewards, learning_gaze, n_train, use_gaze, snippet_length)
print("num training_obs", len(training_obs)) # tuple with pairs of trajectories
print("num_labels", len(training_labels))
print((training_obs[0][0][49].shape)) 

num training_obs 10000
num_labels 10000
(1, 84, 84, 4)


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
reward_net = Net()
reward_net.to(device)

Net(
  (conv1): Conv2d(4, 16, kernel_size=(7, 7), stride=(3, 3))
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(2, 2))
  (conv3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)

In [8]:
import torch.optim as optim
optimizer = optim.Adam(reward_net.parameters(),  lr=lr, weight_decay=weight_decay)
learn_reward(reward_net, optimizer, training_obs, training_labels, training_gaze, num_iter, l1_reg, reward_model_path, use_gaze)

with torch.no_grad():
    pred_returns = [predict_traj_return(reward_net, traj) for traj in demonstrations]
for i, p in enumerate(pred_returns):
    print(i,p,sum(learning_rewards[i]),learning_returns[i])

print("accuracy", calc_accuracy(reward_net, training_obs, training_labels))

cuda:0
training data:  <class 'tuple'>
output loss:  tensor(0.6927, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.6768672315038102
total loss:  tensor([[1.0311]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6958, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.7797442938248115
total loss:  tensor([[1.0856]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6934, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.7461098037874097
total loss:  tensor([[1.0665]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6930, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT

output loss:  tensor(0.6905, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.8088077052625267
total loss:  tensor([[1.0949]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7046, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.8067486489028595
total loss:  tensor([[1.1079]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6954, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.7729246445687622
total loss:  tensor([[1.0818]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6825, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv ma

output loss:  tensor(0.6973, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.7023048460395116
total loss:  tensor([[1.0484]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6989, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.775386191315619
total loss:  tensor([[1.0866]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6856, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.735716421585696
total loss:  tensor([[1.0535]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6708, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map 

output loss:  tensor(0.6818, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.6195645583984939
total loss:  tensor([[0.9916]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7019, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.5787151085096585
total loss:  tensor([[0.9913]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6516, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.6332429930962326
total loss:  tensor([[0.9682]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7605, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv ma

output loss:  tensor(0.6214, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.42831941463414724
total loss:  tensor([[0.8355]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7776, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.45985310054877454
total loss:  tensor([[1.0075]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.6764, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4157782115052876
total loss:  tensor([[0.8843]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7735, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv 

...

output loss:  tensor(0.6170, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.42532833212267973
total loss:  tensor([[0.8297]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.2097, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.44644302526461827
total loss:  tensor([[0.4329]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(1.0293, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.460308761204083
total loss:  tensor([[1.2595]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.8727, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv m

output loss:  tensor(0.2809, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4852178032556004
total loss:  tensor([[0.5235]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.7279, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.3888450202823437
total loss:  tensor([[0.9223]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.5885, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.391423028464718
total loss:  tensor([[0.7842]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.4216, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map

output loss:  tensor(0.8855, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4085670908825696
total loss:  tensor([[1.0898]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(1.4797, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4106647940965932
total loss:  tensor([[1.6850]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(1.1458, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4197697488584796
total loss:  tensor([[1.3557]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.1502, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv ma

output loss:  tensor(0.8752, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.41490430853539584
total loss:  tensor([[1.0827]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.2205, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.40172328832714027
total loss:  tensor([[0.4214]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(1.0575, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv map shape:  (50, 1, 7, 7)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
gaze loss:  0.4035595776985036
total loss:  tensor([[1.2593]], device='cuda:0', grad_fn=<AddBackward0>)
output loss:  tensor(0.3653, device='cuda:0', grad_fn=<NllLoss2DBackward>)
1
GT gaze map shape:  (50, 1, 7, 7)
conv 

NameError: name 'reward_net' is not defined